<a href="https://colab.research.google.com/github/chintan1529/Brain-Tumor-mri-prediction-/blob/main/DLModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

dataset_path = "/content/drive/MyDrive/archive"
print("Folders inside dataset path:", os.listdir(dataset_path))


Folders inside dataset path: ['brain_tumor_dataset']


In [6]:


# Step 2: Import Required Libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 3: Correct Dataset Path
dataset_path = "/content/drive/MyDrive/archive/brain_tumor_dataset"

# Step 4: Load and Preprocess Images
img_size = 150  # Resize all images to 150x150

def load_data(directory):
    images = []
    labels = []
    class_labels = {"yes": 1, "no": 0}

    for class_name, label in class_labels.items():
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            print(f"Loading images from: {class_path}")  # Debugging
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                    img = cv2.resize(img, (img_size, img_size))
                    images.append(img)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels)

# Load dataset
X, y = load_data(dataset_path)

# Check if images loaded correctly
if X.shape[0] == 0:
    print("No images found! Check the dataset path.")

# Normalize images
X = X / 255.0

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Define the CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: Tumor (1) / No Tumor (0)
])

# Step 6: Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 7: Train the Model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Step 8: Evaluate the Model
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {acc*100:.2f}%")

# Step 9: Save Model
model.save('/content/drive/MyDrive/brain_tumor_model.h5')
print("Model saved successfully!")


Loading images from: /content/drive/MyDrive/archive/brain_tumor_dataset/yes
Loading images from: /content/drive/MyDrive/archive/brain_tumor_dataset/no


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6767 - loss: 1.0878 - val_accuracy: 0.7059 - val_loss: 0.6388
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7628 - loss: 0.5778 - val_accuracy: 0.7059 - val_loss: 0.5780
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.8113 - loss: 0.4681 - val_accuracy: 0.7451 - val_loss: 0.5621
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8104 - loss: 0.4596 - val_accuracy: 0.7647 - val_loss: 0.5517
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7935 - loss: 0.4367 - val_accuracy: 0.7451 - val_loss: 0.6413
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8357 - loss: 0.4088 - val_accuracy: 0.7451 - val_loss: 0.6084
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8272 - loss: 0.4149 - val_accuracy: 0.7451 - val_loss: 0.5507
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8583 - loss: 0.3379 - val_accuracy: 0.7451 - val_loss: 0.5428
Epoch 9/10
7

Validation Accuracy: 84.31%
Model saved successfully!


In [14]:
!pip install streamlit --quiet
!pip install pyngrok --quiet
!ngrok authtoken Your_auth_token  # Replace with your ngrok token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/brain_tumor_model.h5')

def predict_tumor(image):
    img = cv2.resize(image, (150, 150))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    result = "Tumor Detected" if prediction > 0.5 else "No Tumor"
    return result, prediction

def main():
    st.title("🧠 Brain Tumor Detection")
    st.write("Upload an MRI scan, and the model will predict whether a brain tumor is present.")

    uploaded_file = st.file_uploader("Choose an MRI Image...", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded MRI Image", use_column_width=True)

        image = np.array(image)
        if image.shape[-1] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

        result, score = predict_tumor(image)
        st.write(f"### Prediction: {result}")
        st.write(f"### Confidence Score: {score:.4f}")

if __name__ == '__main__':
    main()


Overwriting app.py


In [17]:
from pyngrok import ngrok
import os
import time

# Kill any existing ngrok tunnels
os.system("pkill -f ngrok")

# Start Streamlit
os.system("streamlit run app.py &")

# Wait for Streamlit to start
time.sleep(5)

# Correct way to expose port 8501 using ngrok
public_url = ngrok.connect(8501)  # No need for "port="
print(f"🚀 Streamlit app is running at: {public_url}")


🚀 Streamlit app is running at: NgrokTunnel: "https://be66-104-155-226-102.ngrok-free.app" -> "http://localhost:8501"
